In [5]:
import pandas as pd
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.metrics import mean_squared_error
from sklearn.preprocessing import StandardScaler, RobustScaler, LabelEncoder
import csv
import xgboost as xgb
import numpy as np

In [6]:
df = pd.read_csv('train.csv')
df.drop_duplicates(inplace=True)
X = df.loc[:, df.columns != 'price_doc']
y = df['price_doc']

object_cols = X.select_dtypes(include=['object']).columns
label = LabelEncoder()

for col in object_cols:
    X[col] = label.fit_transform(X[col].astype(str))

X = np.log1p(X)
num_cols = X.select_dtypes(include=['int64','float64']).columns

scaler = RobustScaler()
X[num_cols] = scaler.fit_transform(X[num_cols])

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)


C:\Users\Irtiza\AppData\Local\Temp\ipykernel_11144\1714889007.py:10: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  X[col] = label.fit_transform(X[col].astype(str))
C:\Users\Irtiza\AppData\Local\Temp\ipykernel_11144\1714889007.py:10: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  X[col] = label.fit_transform(X[col].astype(str))
C:\Users\Irtiza\AppData\Local\Temp\ipykernel_11144\1714889007.py:10: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_inde

In [8]:
df = pd.read_csv('train.csv')
df.drop_duplicates(inplace=True)

X = df.loc[:, df.columns != 'price_doc']
y = df['price_doc']

object_cols = X.select_dtypes(include=['object']).columns
label = LabelEncoder()

for col in object_cols:
    X[col] = label.fit_transform(X[col].astype(str))

X = np.log1p(X)
num_cols = X.select_dtypes(include=['int64', 'float64']).columns

scaler = RobustScaler()  # Change to RobustScaler
X[num_cols] = scaler.fit_transform(X[num_cols])

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)

df_test = pd.read_csv('test.csv')
df_test = df_test.drop(['row ID'], axis=1)

label_test = LabelEncoder()  # Define label_test
for col in object_cols:
    df_test[col] = label_test.fit_transform(df_test[col].astype(str))

df_test = np.log1p(df_test)
df_test[num_cols] = scaler.transform(df_test[num_cols])

# Create XGBoost model
xg_reg = xgb.XGBRegressor()

C:\Users\Irtiza\AppData\Local\Temp\ipykernel_11144\2055335651.py:11: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  X[col] = label.fit_transform(X[col].astype(str))
C:\Users\Irtiza\AppData\Local\Temp\ipykernel_11144\2055335651.py:11: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  X[col] = label.fit_transform(X[col].astype(str))
C:\Users\Irtiza\AppData\Local\Temp\ipykernel_11144\2055335651.py:11: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_inde

In [9]:
# Define the parameter grid for Grid Search
param_grid = {
    'learning_rate': [0.01, 0.1, 0.2],
    'max_depth': [3, 5, 7],
    'n_estimators': [50, 100, 200],
    'colsample_bytree': [0.3, 0.5, 0.7]
}

# Initialize Grid Search with cross-validation
grid_search = GridSearchCV(estimator=xg_reg, param_grid=param_grid, scoring='neg_mean_squared_error', cv=3)

# Fit the grid search to the data
grid_search.fit(X_train, y_train)


GridSearchCV(cv=3,
             estimator=XGBRegressor(base_score=None, booster=None,
                                    callbacks=None, colsample_bylevel=None,
                                    colsample_bynode=None,
                                    colsample_bytree=None, device=None,
                                    early_stopping_rounds=None,
                                    enable_categorical=False, eval_metric=None,
                                    feature_types=None, gamma=None,
                                    grow_policy=None, importance_type=None,
                                    interaction_constraints=None,
                                    learning_rate=None, m...
                                    max_cat_to_onehot=None, max_delta_step=None,
                                    max_depth=None, max_leaves=None,
                                    min_child_weight=None, missing=nan,
                                    monotone_constraints=None,
                                    multi_strategy=None, n_estimators=None,
                                    n_jobs=None, num_parallel_tree=None,
                                    random_state=None, ...),
             param_grid={'colsample_bytree': [0.3, 0.5, 0.7],
                         'learning_rate': [0.01, 0.1, 0.2],
                         'max_depth': [3, 5, 7],
                         'n_estimators': [50, 100, 200]},
             scoring='neg_mean_squared_error')

In [10]:
# Get the best parameters from the grid search
best_params = grid_search.best_params_
print(best_params)

{'colsample_bytree': 0.5, 'learning_rate': 0.1, 'max_depth': 5, 'n_estimators': 50}


In [11]:
# Use the best parameters to create the final model
final_model = xgb.XGBRegressor(**best_params)
final_model.fit(X_train, y_train)

# Predictions on the test set
y_pred = final_model.predict(df_test)
test_pred = final_model.predict(X_test)

print("Root Mean Squared Error:", mean_squared_error(y_test, test_pred, squared=False))

Root Mean Squared Error: 12918362.034043433


In [13]:
filepath = 'regress_4dec_2.csv'

with open(filepath, mode='w', newline='') as file:
    writer = csv.writer(file)
    c = 1
    writer.writerow(['row ID', 'price_doc'])
    for i in y_pred:
        writer.writerow([c, i])
        c += 1